## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [82]:
df=pd.read_csv('/fake-news/train.csv')

In [83]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [84]:
###Drop Nan Values
df=df.dropna()


In [85]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [86]:
## Get the Dependent features
y=df['label']

In [87]:
X.shape

(18285, 4)

In [88]:
y.shape

(18285,)

In [89]:
import tensorflow as tf

In [90]:
tf.__version__

'2.6.0'

In [91]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [92]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [93]:
messages=X.copy()

In [94]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [95]:
messages.reset_index(inplace=True)

In [96]:
import nltk
import re
from nltk.corpus import stopwords

In [97]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [98]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [99]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [100]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1029, 2577, 3274, 2270, 299, 1678, 3999, 678, 4640, 3040],
 [2907, 726, 3270, 1864, 4756, 1793, 4822],
 [2344, 812, 3149, 4878],
 [1465, 521, 401, 2396, 3462, 2127],
 [567, 4756, 1717, 401, 2656, 3813, 4756, 1719, 279, 1349],
 [3454,
  311,
  467,
  2922,
  3194,
  4576,
  578,
  4674,
  3667,
  413,
  278,
  1850,
  3414,
  872,
  4822],
 [4987, 633, 592, 38, 2347, 4351, 315, 1965, 3511, 1367, 979],
 [4494, 2098, 1896, 2125, 4502, 1404, 4576, 1699, 3511, 1367, 979],
 [3767, 4555, 1302, 512, 1751, 2932, 2381, 998, 4576, 1116],
 [4341, 3815, 2112, 3845, 4268, 3813, 2934, 28],
 [4333, 4746, 1640, 2571, 1765, 2590, 4333, 170, 333, 2970, 1149],
 [2396, 33, 299, 2932, 4576, 4502],
 [1363, 1830, 2714, 4178, 4712, 363, 3122, 3834, 723],
 [1429, 3159, 1372, 1008, 4670, 864, 2250, 3511, 1367, 979],
 [4679, 3147, 4657, 637, 479, 3511, 1367, 979],
 [108, 4021, 653, 2721, 3376, 1012, 1271, 3547, 3047, 274],
 [4504, 2875, 726],
 [4733, 1427, 4657, 3245, 4576, 2102, 1055, 4822],
 [2523, 1763, 3270

### Embedding Representation

In [101]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  678 4640 3040]
 [   0    0    0 ... 4756 1793 4822]
 [   0    0    0 ...  812 3149 4878]
 ...
 [   0    0    0 ... 3511 1367  979]
 [   0    0    0 ...  723  628 4894]
 [   0    0    0 ... 2670 3446 4192]]


In [102]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1029,
       2577, 3274, 2270,  299, 1678, 3999,  678, 4640, 3040], dtype=int32)

In [103]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [104]:
len(embedded_docs),y.shape

(18285, (18285,))

In [105]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [106]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [107]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [108]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 36ms/step - loss: 0.3513 - accuracy: 0.8322 - val_loss: 0.1953 - val_accuracy: 0.9147
Epoch 2/10
192/192 [==============================] - 6s 34ms/step - loss: 0.1446 - accuracy: 0.9407 - val_loss: 0.1916 - val_accuracy: 0.9185
Epoch 3/10
192/192 [==============================] - 6s 34ms/step - loss: 0.1049 - accuracy: 0.9615 - val_loss: 0.2351 - val_accuracy: 0.9181
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0766 - accuracy: 0.9729 - val_loss: 0.2461 - val_accuracy: 0.9167
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0567 - accuracy: 0.9824 - val_loss: 0.2559 - val_accuracy: 0.9118
Epoch 6/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0383 - accuracy: 0.9883 - val_loss: 0.3425 - val_accuracy: 0.9115
Epoch 7/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0284 - accuracy: 0.9906 - val_loss: 0.4307 - val_accuracy: 0.9155

### Adding Dropout 

In [109]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [115]:
model.fit(X_train,y_train,epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 38ms/step - loss: 0.3508 - accuracy: 0.8378
Epoch 2/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1553 - accuracy: 0.9375
Epoch 3/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1094 - accuracy: 0.9598
Epoch 4/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0777 - accuracy: 0.9724
Epoch 5/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0554 - accuracy: 0.9803
Epoch 6/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0400 - accuracy: 0.9871
Epoch 7/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0285 - accuracy: 0.9907
Epoch 8/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0242 - accuracy: 0.9921
Epoch 9/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0222 - accuracy: 0.9924
Epoch 10/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0173 - accura

### Performance Metrics And Accuracy

In [116]:
y_pred=(model.predict(X_test) > 0.5).astype("int32")
#y_classes=np.round(y_pred).astype(int)

In [117]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int32)

In [118]:
from sklearn.metrics import confusion_matrix

In [119]:
confusion_matrix(y_test,y_pred)

array([[3113,  306],
       [ 201, 2415]])

In [120]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.915990057995029